# Quickstart

In [1]:
import numpy as np
from autoemulate.compare import AutoEmulate
from autoemulate.experimental_design import LatinHypercube
from autoemulate.demos.projectile import simulator

In [2]:
# sample from a simulation
lhd = LatinHypercube([(-5., 1.), (0., 1000.)])
X = lhd.sample(100)
y = np.array([simulator(x) for x in X])

In [3]:
# compare emulator models
ae = AutoEmulate()
ae.setup(X, y)
ae.compare()

autoemulate - Cross-validating SecondOrderPolynomial...
autoemulate - Parameters: {'degree': 2}
autoemulate - Cross-validating RBF...
autoemulate - Parameters: {'degree': 1, 'epsilon': 1.0, 'kernel': 'thin_plate_spline', 'smoothing': 0.0}
autoemulate - Cross-validating RandomForest...
autoemulate - Parameters: {'bootstrap': True, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_samples': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'oob_score': False, 'random_state': None}
autoemulate - Cross-validating GradientBoosting...
autoemulate - Parameters: {'ccp_alpha': 0.0, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'n_iter_no_change': None, 'random_state': None, 'subsample': 1.0}
autoemulate - Cross-validating GaussianProcessSk...
autoemulate - Parameters: {'alpha': 1e-10, 'copy_X_train': True, 'kernel__length_scale': 1.0, 'kerne

Pipeline(steps=[('scaler', StandardScaler()), ('model', GaussianProcessSk())])

In [4]:
# evaluate
ae.print_results()

Average scores across all models:
metric                  model        r2          rsme
0           GaussianProcessSk  0.999307    106.905986
1            GradientBoosting  0.891513   2689.064412
2                 NeuralNetSk -0.188991  10036.524609
3              NeuralNetTorch -0.270703   9473.533594
4                         RBF  0.994113    635.634626
5                RandomForest  0.871152   2777.605641
6       SecondOrderPolynomial -1.153115   4564.968893
7       SupportVectorMachines  0.806421   3387.518778
8                     XGBoost  0.825064   3223.579721


In [5]:
# save & load best model
ae.save_model("best_model")
best_emulator = ae.load_model("best_model")

In [6]:
# emulate
best_emulator.predict(X)

array([1.20696211e+02, 1.32714024e+03, 2.30839243e+00, 4.81732472e+00,
       2.89100418e+02, 1.10528887e+03, 8.37770052e+03, 7.91997748e-01,
       1.29787993e+04, 5.52689829e+03, 9.49288961e+03, 2.21286786e+02,
       1.04503203e+02, 5.42992738e+00, 1.90118757e+02, 2.17869938e+04,
       2.68323326e+04, 8.26112724e+01, 8.61463698e+00, 5.27173308e+02,
       7.14924466e+01, 1.95099924e+00, 6.21735193e+01, 1.42147400e+04,
       8.41307341e-01, 1.63223838e+00, 2.81177101e+04, 1.21330949e+00,
       9.60668811e+00, 2.59526453e+04, 4.44748254e+00, 1.89211337e+02,
       5.66044798e+03, 5.96367690e+01, 2.54262685e+03, 1.03527671e+02,
       2.13155469e+04, 2.77288057e+00, 1.07966457e+01, 1.85925564e+00,
       3.97153334e+01, 1.30476590e+02, 1.12610537e+03, 1.63727947e+02,
       3.12093586e+03, 6.63696230e+03, 1.00382742e+02, 7.27967087e+00,
       8.26231148e+00, 2.17770913e+01, 1.75216354e+01, 8.55556410e+01,
       6.28324517e-01, 1.09005891e+01, 1.54515971e+01, 1.67813176e+04,
      